In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import math
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn import mixture

### Functions

In [104]:
#PCA get principal components Functions
def getEigens(X):
    pca = PCA(n_components = d)
    pca.fit_transform(X)
    print(pca.explained_variance_ratio_)
    return pca.explained_variance_ratio_, pca.components_

def getReduced(X,r):
  pca = PCA(n_components = r)
  pca.fit(X)
  X_reduced = pca.transform(X)
  return X_reduced

# Identifying Components with small Eigenvalue
THRESHOLD = 0.001
def findNormalVectors(eigens,vectors):
  normal_vectors = []
  for i in range(0,len(eigens)):
    if eigens[i] > THRESHOLD:
      continue
    normal_vectors.append(vectors[j])
  return normal_vectors

def normalized(X, centroids):
  for i in range(0,len(X)):
    X.iloc[i]-= centroids
  return preprocessing.normalize(X)

### Open Input

In [105]:
# Input
subtask = 2
file = open(f'./R2{subtask}.txt','r')

d,n,m,k,p = list(map(int,file.readline().split()))
k_list = list(map(int,file.readline().split()))
ar=[]
for i in range(0,n):
  ar.append(list(map(float,file.readline().split())))
df= pd.DataFrame(ar)

#Visualization in 3D
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(111, projection='3d')
x = np.array(df[0])
y = np.array(df[1])
z = np.array(df[2])
img = ax.scatter(x, y, z, c='green')
plt.show()

<IPython.core.display.Javascript object>

### Parameters

In [106]:
def showInertia(X):
  num_clusters = 12
  kmeans_tests = [KMeans(n_clusters=i, init='random', n_init=10) for i in range(2, num_clusters)]
  inertia = [kmeans_tests[i].fit(X).inertia_ for i in range(len(kmeans_tests))]
  plt.plot(range(2, num_clusters),inertia,'gs-')
  plt.xlabel('Number of Clusters')
  plt.ylabel('Intertia')
  plt.title('Elbow Curve')
  plt.show()

In [107]:
def showScore(X):
  num_clusters = 12
  kmeans_tests = [KMeans(n_clusters=i, init='random', n_init=10) for i in range(2, num_clusters)]
  score = [kmeans_tests[i].fit(X).score(X) for i in range(len(kmeans_tests))]
  plt.plot(range(2, num_clusters),score,'gs-')
  plt.xlabel('Number of Clusters')
  plt.ylabel('Score')
  plt.title('Elbow Curve')
  plt.show()

In [108]:
from sklearn.metrics import silhouette_score
def showSilouette(X, model_list):
  silouette = [silhouette_score(X,model_list[i].fit(X).labels_) for i in range(len(model_list))]
  plt.plot(range(2, len(model_list)+2),silouette,'gs-')
  plt.xlabel('Number of Clusters')
  plt.ylabel('Silhouette Score')
  plt.title('Elbow Curve')
  plt.show()

In [109]:
def create_Hierarchical_Models(num_clusters):
  return [AgglomerativeClustering(n_clusters=i, affinity='euclidean') for i in range(2, num_clusters)]
def create_KMeans_Models(num_clusters):
  return [KMeans(n_clusters=i, init='random', n_init=10) for i in range(2, num_clusters)]

In [110]:
def getMaxSilouette(X, model_list):
  index_silouette = [(i,silhouette_score(X,model_list[i].fit(X).predict(X))) for i in range(len(model_list))]
  return max(index_silouette, key=lambda p: p[1])[0]

In [111]:
# Epsilon for DBSCAN
from sklearn.neighbors import NearestNeighbors
def show_KDistance_Graph(X):
  neigh = NearestNeighbors(n_neighbors=5)
  nbrs = neigh.fit(X)
  distances, indices = nbrs.kneighbors(X)
  # Plotting K-distance Graph
  distances = np.sort(distances, axis=0)
  distances = distances[:,1]
  print(distances[-30:])
  plt.figure(figsize=(20,10))
  plt.plot(distances)
  plt.title('K-distance Graph',fontsize=20)
  plt.xlabel('Data Points sorted by distance',fontsize=14)
  plt.ylabel('Epsilon',fontsize=14)
  plt.show()

In [112]:
show_KDistance_Graph(df)

[ 9.20486061  9.25129212  9.25129212  9.26138508  9.28929041  9.28929041
  9.31667106  9.33198855  9.42608811  9.42807574  9.45673179  9.52123069
  9.52123069  9.5726656   9.78716655  9.82459821  9.86526388  9.91624076
  9.92415326  9.95000042  9.96458979 10.03527935 10.04185947 10.04358057
 10.10767148 10.11232332 10.22709081 10.33413966 10.4315936  10.75807412]


<IPython.core.display.Javascript object>

### Finding Submanifolds

In [113]:
#DBSCAN
from sklearn.cluster import DBSCAN
EPS = 28
MIN_SAMPLES = 2 * d +2
model_name = 'dbscan'
model = DBSCAN(eps=EPS, min_samples=MIN_SAMPLES)
model.fit(df[[i for i in range(0,d)]])
df['manifold'] = model.labels_

### Manifold Visualization

In [114]:
#Manifold Visualization in Higher Dimension
print(f'model name: {model_name}')
for i in range(0,d):
  fig = plt.figure(figsize=(10, 10))
  ax = fig.add_subplot(111, projection='3d')
  X = df[[i for i in range(0,d)]]
  x = np.array(X[i%d])
  y = np.array(X[(i+1)%d])
  z = np.array(X[(i+2)%d])
  ax.scatter(x,y,z, marker="s", c=df['manifold'], s=2, cmap="Set1")
  # ax.view_init(90, angle)
  plt.show()

model name: dbscan


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Finding Balls

In [115]:
#Miniball Method
import miniball
centroids = [0 for i in range(m)]
radius = [0]*m
for i in range(m):
  points = df.loc[df['manifold'] == i][[i for i in range(0,d)]].to_numpy()
  minlist = [min(points, key=lambda p: p[j])[j] for j in range(0,d)]
  points = [[p[j]-minlist[j] for j in range(0,d)] for p in points]
  mb = miniball.Miniball(points)
  c = mb.center()
  for j in range(0,d):
    c[j]+=minlist[j]
  r = math.sqrt(mb.squared_radius())
  centroids[i] = c
  radius[i] = r
  if not mb.is_valid():
    print('Possibly invalid!')
  print('Relative error', mb.relative_error())

Relative error 1.1238281227599984e-15
Relative error 1.3078051727382375e-15
Relative error 1.392179700011672e-15


### Normal Vectors

#### PCA method

In [116]:
#PCA get principal components
values = []
vectors = []
for i in range(0,m):
    X = df.loc[df['manifold'] == i][[i for i in range(0,d)]]
    s,v = getEigens(X)
    values.append(s)
    vectors.append(v)

[0.2438548  0.19237488 0.15087525 0.10092871 0.08146707 0.06548093
 0.05981207 0.04156986 0.03383309 0.02980335]
[7.60262636e-01 2.39737364e-01 8.17974779e-31 3.61037075e-32
 2.77254637e-32 1.59161176e-32 8.23180855e-33 7.14144205e-33
 1.97921471e-33 8.52058713e-34]
[2.76524894e-01 2.50236445e-01 1.92229897e-01 1.50202504e-01
 1.30806260e-01 1.98565605e-31 6.47016927e-32 2.47331636e-32
 1.36193266e-32 4.20934266e-33]


In [117]:
#Finding Normal Vectors using PCA
A = []
for i in range(0,m):
  A.append(findNormalVectors(values[i],vectors[i]))

In [118]:
#Finding b-s using PCA
B = []
for i in range(0,m):
  b = []
  X = df.loc[df['manifold'] == i][[i for i in range(0,d)]]
  mean = np.mean(X, axis = 0)
  for j in range(0,len(A[i])):
    b.append(np.dot(A[i][j],mean))
  B.append(b)

In [119]:
D = [(d-len(A[i])) for i in range(m)]

#### Numerical Error PCA

In [120]:
# Checking Orthogonality (PCA)
Avg_Error_Span_PCA = []
for i in range(0,m):
  data_list_avg=[0 for i in range(0,len(vectors[i])*(len(vectors[i])-1))]
  for j in range(0,len(vectors[i])):
    for s in range(0,len(vectors[i])):
      if j != s:
        data_list_avg.append(np.dot(vectors[i][j],vectors[i][s]))
  Avg_Error_Span_PCA.append(sum(data_list_avg)/len(data_list_avg))
print(Avg_Error_Span_PCA)

[1.9621649983826553e-17, 3.0526314500869646e-18, -3.450172246664983e-18]


In [121]:
#Checking Orthogonality Error (PCA)
Avg_Error_Total_PCA = []
for i in range(0,m):
  X = df.loc[df['manifold'] == i][[s for s in range(0,d)]]
  mean = np.mean(X, axis = 0)
  data_list_avg=[0 for i in range(0,len(X))]
  for j in range(0,len(X)):
    v = [np.dot(A[i][s],X.iloc[j])-B[i][s] for s in range(0,len(A[i]))]
  if len(v)!=0:
    data_list_avg.append(sum(v)/len(v))
  Avg_Error_Total_PCA.append(sum(data_list_avg)/len(data_list_avg))
print(Avg_Error_Total_PCA)

[0.0, 0.0, 0.0]


### Match k_i's with manifolds

In [122]:
# Plot Elbow
print(D)
num_clusters=12
for i in range(m):
  print(i)
  X = df.loc[df['manifold'] == i][[i for i in range(0,d)]]
  showSilouette(X,create_KMeans_Models(num_clusters))
  showSilouette(X,create_Hierarchical_Models(num_clusters))

[10, 2, 5]
0
1
2


In [123]:
if subtask == 1:
  final_k_list = [1, 1, 1]
elif subtask == 2:
  final_k_list = [5, 5, 3]
print(final_k_list)

[5, 5, 3]


### Clustering

In [124]:
# Joining the cluster labeling
def join_clusters():
  df['cluster'] = [0] * n
  data_index = [0] * m
  for i in range(n):
    index_manifold = df.at[i,'manifold']
    df.at[i,'cluster'] = list_labels[index_manifold][data_index[index_manifold]]
    data_index[index_manifold] += 1

model_name = 'k-means'
list_labels = []
if model_name == 'k-means':
  for i in range(0,m):
    model=KMeans(n_clusters=final_k_list[i], n_init=1000, max_iter=1500)
    X = df.loc[df['manifold'] == i][[i for i in range(0,d)]]
    model.fit(X)
    list_labels.append(model.labels_)
elif model_name == 'heirarchical':
  for i in range(0,m):
    model_name = 'heirarchical'
    model = AgglomerativeClustering(n_clusters=final_k_list[i], affinity='cosine', linkage = 'average')
    X = df.loc[df['manifold'] == i][[i for i in range(0,d)]]
    model.fit(normalized(X,centroids[i]))
    list_labels.append(model.labels_)
elif model_name == 'GMM':
  for i in range(0,m):
    X = df.loc[df['manifold'] == i][[i for i in range(0,d)]]
    model = mixture.GaussianMixture(n_components=final_k_list[i], covariance_type='full', n_init=200)
    model.fit(X)
    labels = model.predict(X)
    list_labels.append(labels)
join_clusters()

### Clustering Visualization

In [125]:
# Clustering Visualization in 3D
%matplotlib notebook
print(f'model name: {model_name}')
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
x = np.array(df[0])
y = np.array(df[1])
z = np.array(df[2])
img = ax.scatter(x, y, z, c=df['cluster'] + 3 * df['manifold'])
plt.show()

model name: k-means


<IPython.core.display.Javascript object>

### Output

In [126]:
# Outputting
vectors = [[[] for j in range(0,final_k_list[i])] for i in range(0,m)]
outlier = []
for i in range(n):
  if df.at[i, 'cluster'] == -1:
    outlier.append(i + 1)
  else:
    vectors[df.at[i, 'manifold']][df.at[i, 'cluster']].append(i + 1)

with open('output.txt', 'w') as f:
  print(f'{n} {m}', file=f)
  for i in range(m):
    dimension = d-len(A[i])
    print(f'{dimension} ' + str(final_k_list[i]) + ' Sphere', file=f)
    for j in range(0,len(A[i])):
      print(' '.join(list(map(str,A[i][j]))), file=f)
    print(' '.join(list(map(str,B[i]))), file=f)
    print(' '.join(list(map(str, centroids[i]))) + ' ' + str(radius[i]), file=f)
    for j in range(0,final_k_list[i]):
      print(str(len(vectors[i][j])) + ' ' + ' '.join(list(map(str, vectors[i][j]))), file=f)
  print(str(len(outlier)) + ' ' + ' '.join(list(map(str, outlier))), file=f)

